In [ ]:
import scipy.io
import numpy as np
from sklearn.metrics import auc
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statistics import variance, mean

In [ ]:
hc_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/attacks240/attack_HC/clin_attack_HC_06-11-2020 16-34v4.mat'
st0_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/attacks240/attack_ST01/clin_attack_ST01_06-10-2020 16-06v4.mat'
st1_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/attacks240/attack_ST02/clin_attack_ST02_06-11-2020 16-18v4.mat'
st2_data_path = '/Users/jk1/unige_onedrive/OneDrive - unige.ch/BCT/attacks240/attack_ST03/clin_attack_ST03_06-11-2020 16-26v4.mat'

In [ ]:
# global eff after attack
outcome_var = 'GlobEff_bin_new'
pre_attack_outcome_var = 'GlobEff_bin'
minimum_connectivity_threshold = 0.3

For this variable, data is available for the 8 remaining threshold bins [0.3-1.0]

In [ ]:
hc_data_mat = scipy.io.loadmat(hc_data_path)
st0_data_mat = scipy.io.loadmat(st0_data_path)
st1_data_mat = scipy.io.loadmat(st1_data_path)
st2_data_mat = scipy.io.loadmat(st2_data_path)

In [ ]:
n_hc = len(hc_data_mat[outcome_var]['C'][0][0])
n_st = len(st0_data_mat[outcome_var]['C'][0][0]) + 1 # accounting for missing patient

all_subj_idx = np.arange(n_hc + n_st)
hc_idx = all_subj_idx[:n_hc]
st_idx = all_subj_idx[n_hc:]

In [ ]:
# use auc only over predefined area of thresholds
# for glob eff after attack threshold range is already preselected
def custom_auc(values_over_thresholds):
    connectivity_thresholds = np.arange(minimum_connectivity_threshold, 1.1, 0.1)
    return auc(connectivity_thresholds, values_over_thresholds)


In [ ]:
def extract_glob_eff_auc_after_attack(data_mat, n_subj, group, subj_idx, timepoint):
    # !!!!!!! glob_eff_pre_attack is computed differently than in longitudinal analysis
    glob_eff_pre_attack = np.vstack(data_mat[pre_attack_outcome_var][0][0]).T

    glob_eff_cortical_attack= \
        np.squeeze(
                        [np.vstack(data_mat[outcome_var]['C'][0][0][i])
                         for i in range(len(data_mat[outcome_var]['C'][0][0]))]
                    )

    glob_eff_cortico_subcortical_attack= \
        np.squeeze(
                        [np.vstack(data_mat[outcome_var]['CSUB'][0][0][i])
                         for i in range(len(data_mat[outcome_var]['C'][0][0]))]
                    )

    glob_eff_subcortical_attack= \
        np.squeeze(
                        [np.vstack(data_mat[outcome_var]['SUB'][0][0][i])
                         for i in range(len(data_mat[outcome_var]['C'][0][0]))]
                    )
    # correct for missing values - (pt1 TP1, pt5 Tp2, Pt13 (=pt17) TP3)
    if group == 'st':
        n_bins = glob_eff_cortical_attack.shape[1]
        if timepoint == 0:
            glob_eff_pre_attack = np.insert(glob_eff_pre_attack, 0, np.full((n_bins), np.NaN), axis=0)
            glob_eff_cortical_attack = np.insert(glob_eff_cortical_attack, 0, np.full((n_bins), np.NaN), axis=0)
            glob_eff_cortico_subcortical_attack = np.insert(glob_eff_cortico_subcortical_attack, 0, np.full((n_bins), np.NaN), axis=0)
            glob_eff_subcortical_attack = np.insert(glob_eff_subcortical_attack, 0, np.full((n_bins), np.NaN), axis=0)
        if timepoint == 1:
            glob_eff_pre_attack = np.insert(glob_eff_pre_attack, 4, np.full((n_bins), np.NaN), axis=0)
            glob_eff_cortical_attack = np.insert(glob_eff_cortical_attack, 4, np.full((n_bins), np.NaN), axis=0)
            glob_eff_cortico_subcortical_attack = np.insert(glob_eff_cortico_subcortical_attack, 4, np.full((n_bins), np.NaN), axis=0)
            glob_eff_subcortical_attack = np.insert(glob_eff_subcortical_attack, 4, np.full((n_bins), np.NaN), axis=0)
        if timepoint == 2:
            glob_eff_pre_attack = np.insert(glob_eff_pre_attack, 12, np.full((n_bins), np.NaN), axis=0)
            glob_eff_cortical_attack = np.insert(glob_eff_cortical_attack, 12, np.full((n_bins), np.NaN), axis=0)
            glob_eff_cortico_subcortical_attack = np.insert(glob_eff_cortico_subcortical_attack, 12, np.full((n_bins), np.NaN), axis=0)
            glob_eff_subcortical_attack = np.insert(glob_eff_subcortical_attack, 12, np.full((n_bins), np.NaN), axis=0)

    gEff_auc_pre_attack = np.apply_along_axis(custom_auc, arr=glob_eff_pre_attack, axis=-1)
    gEff_auc_cortical_attack = np.apply_along_axis(custom_auc, arr=glob_eff_cortical_attack, axis=-1)
    gEff_auc_cortico_subcortical_attack = np.apply_along_axis(custom_auc, arr=glob_eff_cortico_subcortical_attack, axis=-1)
    gEff_auc_subcortical_attack = np.apply_along_axis(custom_auc, arr=glob_eff_subcortical_attack, axis=-1)

    gEff_auc_cortical_attack_df = pd.DataFrame(gEff_auc_cortical_attack, columns=['glob_eff_auc'])
    gEff_auc_cortical_attack_df['pre_attack_glob_eff_auc'] = gEff_auc_pre_attack
    gEff_auc_cortical_attack_df['subject'] = subj_idx
    gEff_auc_cortical_attack_df['attack_location'] = 'cortical'

    gEff_auc_cortico_subcortical_attack_df = pd.DataFrame(gEff_auc_cortico_subcortical_attack, columns=['glob_eff_auc'])
    gEff_auc_cortico_subcortical_attack_df['pre_attack_glob_eff_auc'] = gEff_auc_pre_attack
    gEff_auc_cortico_subcortical_attack_df['subject'] = subj_idx
    gEff_auc_cortico_subcortical_attack_df['attack_location'] = 'cortico_subcortical'

    gEff_auc_subcortical_attack_df = pd.DataFrame(gEff_auc_subcortical_attack, columns=['glob_eff_auc'])
    gEff_auc_subcortical_attack_df['pre_attack_glob_eff_auc'] = gEff_auc_pre_attack
    gEff_auc_subcortical_attack_df['subject'] = subj_idx
    gEff_auc_subcortical_attack_df['attack_location'] = 'subcortical'

    gEff_auc_df = pd.concat([gEff_auc_cortical_attack_df,gEff_auc_cortico_subcortical_attack_df, gEff_auc_subcortical_attack_df],
                                ignore_index=True)

    gEff_auc_df['attack_type'] = 'clinical'
    gEff_auc_df['group'] = group
    gEff_auc_df['timepoint'] = timepoint

    return gEff_auc_df

In [ ]:
# without reducing to AUC
def extract_glob_eff_after_attack(data_mat, n_subj, group, subj_idx, timepoint):
    glob_eff_cortical_attack= \
        np.squeeze(
                        [np.vstack(data_mat[outcome_var]['C'][0][0][i])
                         for i in range(len(data_mat[outcome_var]['C'][0][0]))]
                    )

    glob_eff_cortico_subcortical_attack= \
        np.squeeze(
                        [np.vstack(data_mat[outcome_var]['CSUB'][0][0][i])
                         for i in range(len(data_mat[outcome_var]['C'][0][0]))]
                    )

    glob_eff_subcortical_attack= \
        np.squeeze(
                        [np.vstack(data_mat[outcome_var]['SUB'][0][0][i])
                         for i in range(len(data_mat[outcome_var]['C'][0][0]))]
                    )

        # correct for missing values - (pt1 TP1, pt5 Tp2, Pt13 (=pt17) TP3)
    if group == 'st':
        n_bins = glob_eff_cortical_attack.shape[1]
        if timepoint == 0:
            glob_eff_cortical_attack = np.insert(glob_eff_cortical_attack, 0, np.full((n_bins), np.NaN), axis=0)
            glob_eff_cortico_subcortical_attack = np.insert(glob_eff_cortico_subcortical_attack, 0, np.full((n_bins), np.NaN), axis=0)
            glob_eff_subcortical_attack = np.insert(glob_eff_subcortical_attack, 0, np.full((n_bins), np.NaN), axis=0)
        if timepoint == 1:
            glob_eff_cortical_attack = np.insert(glob_eff_cortical_attack, 4, np.full((n_bins), np.NaN), axis=0)
            glob_eff_cortico_subcortical_attack = np.insert(glob_eff_cortico_subcortical_attack, 4, np.full((n_bins), np.NaN), axis=0)
            glob_eff_subcortical_attack = np.insert(glob_eff_subcortical_attack, 4, np.full((n_bins), np.NaN), axis=0)
        if timepoint == 2:
            glob_eff_cortical_attack = np.insert(glob_eff_cortical_attack, 12, np.full((n_bins), np.NaN), axis=0)
            glob_eff_cortico_subcortical_attack = np.insert(glob_eff_cortico_subcortical_attack, 12, np.full((n_bins), np.NaN), axis=0)
            glob_eff_subcortical_attack = np.insert(glob_eff_subcortical_attack, 12, np.full((n_bins), np.NaN), axis=0)

    gEff_cortical_attack_df = pd.DataFrame(glob_eff_cortical_attack)
    gEff_cortical_attack_df['subject'] = subj_idx
    gEff_cortical_attack_df = gEff_cortical_attack_df.melt(id_vars=['subject'], var_name='density_bin', value_name='glob_eff')
    gEff_cortical_attack_df['attack_location'] = 'cortical'

    gEff_cortico_subcortical_attack_df = pd.DataFrame(glob_eff_cortico_subcortical_attack)
    gEff_cortico_subcortical_attack_df['subject'] = subj_idx
    gEff_cortico_subcortical_attack_df = gEff_cortico_subcortical_attack_df.melt(id_vars=['subject'], var_name='density_bin', value_name='glob_eff')
    gEff_cortico_subcortical_attack_df['attack_location'] = 'cortico_subcortical'

    gEff_subcortical_attack_df = pd.DataFrame(glob_eff_subcortical_attack)
    gEff_subcortical_attack_df['subject'] = subj_idx
    gEff_subcortical_attack_df = gEff_subcortical_attack_df.melt(id_vars=['subject'], var_name='density_bin', value_name='glob_eff')
    gEff_subcortical_attack_df['attack_location'] = 'subcortical'

    glob_eff_df = pd.concat([gEff_cortical_attack_df,gEff_cortico_subcortical_attack_df, gEff_subcortical_attack_df],
                                ignore_index=True)

    glob_eff_df['density_bin'] += 3 # correct for bins starting at 0.3
    glob_eff_df['attack_type'] = 'clinical'
    glob_eff_df['group'] = group
    glob_eff_df['timepoint'] = timepoint

    return glob_eff_df

In [ ]:
hc_glob_eff_auc_df = extract_glob_eff_auc_after_attack(hc_data_mat, n_hc, 'hc', hc_idx, 0)
st0_glob_eff_auc_df = extract_glob_eff_auc_after_attack(st0_data_mat, n_st, 'st', st_idx, 0)
st1_glob_eff_auc_df = extract_glob_eff_auc_after_attack(st1_data_mat, n_st, 'st', st_idx, 1)
st2_glob_eff_auc_df = extract_glob_eff_auc_after_attack(st2_data_mat, n_st, 'st', st_idx, 2)

In [ ]:
# without reducing to AUC
hc_glob_eff_df = extract_glob_eff_after_attack(hc_data_mat, n_hc, 'hc', hc_idx, 0)
st0_glob_eff_df = extract_glob_eff_after_attack(st0_data_mat, n_st, 'st', st_idx, 0)
st1_glob_eff_df = extract_glob_eff_after_attack(st1_data_mat, n_st, 'st', st_idx, 1)
st2_glob_eff_df = extract_glob_eff_after_attack(st2_data_mat, n_st, 'st', st_idx, 2)


In [ ]:
all_glob_eff_auc_df = pd.concat([hc_glob_eff_auc_df, st0_glob_eff_auc_df, st1_glob_eff_auc_df, st2_glob_eff_auc_df],
                            ignore_index=True)

# convert auc to numeric
all_glob_eff_auc_df['glob_eff_auc'] = all_glob_eff_auc_df['glob_eff_auc'].astype('float')
all_glob_eff_auc_df['pre_attack_glob_eff_auc'] = all_glob_eff_auc_df['pre_attack_glob_eff_auc'].astype('float')

In [ ]:
all_glob_eff_df = pd.concat([hc_glob_eff_df, st0_glob_eff_df, st1_glob_eff_df, st2_glob_eff_df],
                            ignore_index=True)
# convert auc to numeric
all_glob_eff_df['glob_eff'] = all_glob_eff_df['glob_eff'].astype('float')

In [ ]:
ax = sns.catplot(x="timepoint", y="glob_eff_auc", hue="group", col='attack_location',
            kind='box', dodge=True,
            data=all_glob_eff_auc_df, palette="Set3"
                )
ax.fig.suptitle('Global efficiency AUC after attack')
ax.fig.subplots_adjust(top=0.85)
plt.show()

In [ ]:
ax = sns.catplot(x="timepoint", y="pre_attack_glob_eff_auc", hue="group", col='attack_location',
            kind='box', dodge=True,
            data=all_glob_eff_auc_df, palette="Set3"
                )
ax.fig.suptitle('Global efficiency AUC before attack')
ax.fig.subplots_adjust(top=0.85)
plt.show()


In [ ]:
ax = sns.catplot(x="timepoint", y="glob_eff", hue="group", col='attack_location',
            kind='box', dodge=True,
            data=all_glob_eff_df, palette="Set3"
                )
ax.fig.suptitle('Global efficiency after attack')
ax.fig.subplots_adjust(top=0.85)
plt.show()

In [ ]:
all_glob_eff_auc_df.to_csv('glob_eff_auc_after_attack_df.csv', index=False)

In [ ]:
all_glob_eff_df.to_csv('glob_eff_after_attack_df.csv', index=False)